# Jupyter Notebook Terminal Emulator
Adam Johnson, IBM

A fully-functional terminal emulator in a Jupyter notebook. Uses [xterm.js](https://xtermjs.org) for a VT100-compliant Javascript terminal. Instead of an actual WebSocket, the notebook uses the Javascript Jupyter cell execute function `Jupyter.notebook.kernel.execute()` as a channel to communicate between the Python runtime (`TerminalServer`) and browser (`TerminalClient`). `TermalServer` forks a shell and attaches stdin and stdout to the aforementioned channel.

__To start the terminal emulator, choose Cell->Run All and scroll to the bottom of the notebook.__

In [ ]:
import pty, os, tty, termios, time, sys, base64, struct
from fcntl import fcntl, F_GETFL, F_SETFL, ioctl

class TerminalServer:
    def __init__(self):
        pid, self.fd = pty.fork()
        if pid == pty.CHILD:
            # we are in the forked process
            # blow it away and replace with a shell
            os.execvp('sh',['sh'])
        else:
            tty.setraw(self.fd, termios.TCSANOW)
            
            #open the shell process file descriptor as read-write
            self.file = os.fdopen(self.fd,'wb+', buffering=0)
            
            #set the file reads to be nonblocking
            flags = fcntl(self.file, F_GETFL)
            fcntl(self.file, F_SETFL, flags | os.O_NONBLOCK)
            
    def transmit(self,data):
        # data in the "channel" is b64 encoded so that control characters
        # don't get lost
        os.write(self.fd, base64.b64decode(data))
        self.receive()
        
    def receive(self):
        try:
            data = os.read(self.fd, 8192)
        except OSError:
            data = b''
        sys.stdout.write(base64.b64encode(data))
        
    def update_window_size(self, rows, cols):
        #notify that the pty size should change to match xterm.js
        TIOCSWINSZ = getattr(termios, 'TIOCSWINSZ', -2146929561)
        s = struct.pack('HHHH', rows, cols, 0, 0)
        ioctl(self.fd, TIOCSWINSZ, s)
        self.receive()

#create the TerminalServer instance
server = TerminalServer()

In [ ]:
%%javascript
var MAX_POLL_INTERVAL = 1500;
var MIN_POLL_INTERVAL = 100;
var BACKOFF_RATE = 1.8;

function TerminalClient(elem) {

    // require xterm.js
    require.config({
      paths: {
          xterm: '//cdnjs.cloudflare.com/ajax/libs/xterm/2.9.2/xterm.min'
      }
    });
    
    require(['xterm'], function(Terminal) {
        var termArea = this.create_ui(elem);
        this.term = new Terminal({
            rows: 25,
            cols: 100
        });
        this.term.open(termArea[0]);
        
        this.term.on('data', function(data) {
            this.handle_transmit(data);
        }.bind(this));
        
        this.term.on('resize', function() {
            this.handle_resize()
        }.bind(this));
        
        // set title
        this.term.on('title', function(title) {
            this.handle_title(title);
        }.bind(this));
        
        // set the initial size correctly
        this.handle_resize();
        
        // reset the terminal
        this.server_exec('server.transmit(b"' + btoa('\r\nreset\r\nclear\r\n') + '")');
        
        // start polling
        this.curPollInterval = MIN_POLL_INTERVAL;
        this.poll_server();
    
    }.bind(this));    
}

TerminalClient.prototype.create_ui = function(elem) {
    // add xterm stylesheet for formatting
    var xtermCssUrl = 'https://cdnjs.cloudflare.com/ajax/libs/xterm/2.9.2/xterm.min.css'
    $('<link/>', {rel: 'stylesheet', href: xtermCssUrl}).appendTo('head');
    
    this.wrap = $('<div>').appendTo(elem);
    this.wrap.css({
        padding: 10,
        margin: 10,
        border: '1px solid black'
    });
    this.titleBar = $('<div>').css({height: 30}).appendTo(this.wrap);
    this.titleText = $('<div>').html('Shell').css({float: 'left'}).appendTo(this.titleBar);
    this.comIndicator = $('<div>').html('&middot;').css({float: 'left', marginLeft: 10}).hide().appendTo(this.titleBar);
    this.termArea = $('<div>').appendTo(this.wrap);
    return this.termArea;
}

TerminalClient.prototype.update_com_indicator = function() {
    this.comIndicator.show().fadeOut(400);
}

TerminalClient.prototype.server_exec = function(cmd) {
    Jupyter.notebook.kernel.execute(cmd, {
        iopub: {
            output: function(data) {
                this.receive_data_callback(data)
            }.bind(this)
        }
    });
    this.update_com_indicator();
}

TerminalClient.prototype.poll_server = function() {
    this.server_exec('server.receive()');
    clearTimeout(this.termPollTimer);
    this.termPollTimer = setTimeout(function() {
        this.poll_server();
    }.bind(this), this.curPollInterval);
    // gradually back off the polling interval
    this.curPollInterval = Math.min(this.curPollInterval*BACKOFF_RATE, MAX_POLL_INTERVAL);
}
TerminalClient.prototype.receive_data_callback = function(data) {
    var decoded = atob(data.content.text);
    this.term.write(decoded);
}
TerminalClient.prototype.handle_transmit = function(data) {
    // we've had interaction, so reset the timer for the next poll
    // to minPollInterval
    this.curPollInterval = MIN_POLL_INTERVAL;
    
    // transmit data to the server, but b64 encode it
    this.server_exec('server.transmit(b"' + btoa(data) + '")');
}

TerminalClient.prototype.handle_resize = function() {
    this.server_exec('server.update_window_size('+ this.term.rows + ', '+ this.term.cols + ')');
}

TerminalClient.prototype.handle_title = function(title) {
    this.titleText.html(title);
}

// create the TerminalClient instance (only once!)
if (window.terminalClient) {
    delete window.terminalClient;
}
window.terminalClient = new TerminalClient(element)